# Предсказательная модель честности отеля по отзывам. 
Представьте, что вы работаете дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

Вам поставлена задача создать такую модель. 

In [58]:
# импортируем библиотеки

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

# 1. Исследование структуры данных

Первоначальная версия датасета содержит 17 полей со следующей информацией:

- hotel_address — адрес отеля;
- review_date — дата, когда рецензент разместил соответствующий отзыв;
- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
- hotel_name — название отеля;
- reviewer_nationality — страна рецензента;
- negative_review — отрицательный отзыв, который рецензент дал отелю;
- review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
- positive_review — положительный отзыв, который рецензент дал отелю;
- review_total_positive_word_counts — общее количество слов в положительном отзыве.
- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
- total_number_of_reviews — общее количество действительных отзывов об отеле;
- tags — теги, которые рецензент дал отелю;
- days_since_review — количество дней между датой проверки и датой очистки;
- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
- lat — географическая широта отеля;
- lng — географическая долгота отеля.

In [59]:
#Читаем данные и смотрим пример данных
hotels = pd.read_csv('data/hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


1.1 Выведим основную информацию о числе непустых значений в столбцах и их типах в таблице.

In [60]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

1.2 Выведим основную статистическую информацию о столбцах.

In [61]:
hotels.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,lat,lng
count,386803.000000,386803.000000,386803.000000,386803.000000,386803.000000,386803.00000,386803.000000,384355.000000,384355.000000
mean,498.246536,8.397231,18.538988,2743.992042,17.776985,7.17725,8.396906,49.443522,2.823402
std,500.258012,0.547881,29.703369,2316.457018,21.726141,11.05442,1.636090,3.466936,4.579043
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.00000,2.500000,41.328376,-0.369758
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.00000,7.500000,48.214662,-0.143649
50%,342.000000,8.400000,9.000000,2134.000000,11.000000,3.00000,8.800000,51.499981,-0.000250
75%,660.000000,8.800000,23.000000,3613.000000,22.000000,8.00000,9.600000,51.516288,4.834443
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.00000,10.000000,52.400181,16.429233


1.3 Посмотрим на пропуски в данных.

In [62]:
# Вычисляем количество пропусков в каждом столбце
missing_values = hotels.isnull().sum()

# Выводим столбцы с пропусками
print("Пропуски данных в столбцах:")
print(missing_values)

Пропуски данных в столбцах:
hotel_address                                    0
additional_number_of_scoring                     0
review_date                                      0
average_score                                    0
hotel_name                                       0
reviewer_nationality                             0
negative_review                                  0
review_total_negative_word_counts                0
total_number_of_reviews                          0
positive_review                                  0
review_total_positive_word_counts                0
total_number_of_reviews_reviewer_has_given       0
reviewer_score                                   0
tags                                             0
days_since_review                                0
lat                                           2448
lng                                           2448
dtype: int64


Строки с пропусками данных составляют менее 1%. Удалим их.

1.4 Посмотрим на количество уникальных значений в каждом из признаков.

In [63]:
# Получаем количество уникальных значений для каждого признака
unique_values_count = hotels.nunique()

# Выводим результат
print(unique_values_count)

hotel_address                                   1493
additional_number_of_scoring                     480
review_date                                      731
average_score                                     34
hotel_name                                      1492
reviewer_nationality                             225
negative_review                               248828
review_total_negative_word_counts                402
total_number_of_reviews                         1142
positive_review                               311737
review_total_positive_word_counts                354
total_number_of_reviews_reviewer_has_given       194
reviewer_score                                    37
tags                                           47135
days_since_review                                731
lat                                             1472
lng                                             1472
dtype: int64


Видно что количество уникальных значений для hotel_address, hotel_name, lat, lng практически совпадает, что говорит о том, что данные столбцы содержат одинаковую информацию, а значит какие-то из них можно удалить. Так как hotel_address и hotel_name тип данных object, а lat и lng практически полностью дублируют информацию типом данных float64, то в числовой кодировке hotel_address и hotel_name нет необходимости и их можно удалить.

In [64]:
# Удаление столбцов 'hotel_address' и 'hotel_name' в исходном DataFrame
hotels.drop(columns=['hotel_address', 'hotel_name'], inplace=True)

# Проверка результата
print(hotels.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 15 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                386803 non-null  int64  
 1   review_date                                 386803 non-null  object 
 2   average_score                               386803 non-null  float64
 3   reviewer_nationality                        386803 non-null  object 
 4   negative_review                             386803 non-null  object 
 5   review_total_negative_word_counts           386803 non-null  int64  
 6   total_number_of_reviews                     386803 non-null  int64  
 7   positive_review                             386803 non-null  object 
 8   review_total_positive_word_counts           386803 non-null  int64  
 9   total_number_of_reviews_reviewer_has_given  386803 non-null  int64  
 

# 2. Преобразование данных

2.1 Преобразуем строку в 'days_since_review' в число.

In [65]:

# Извлекаем только числовые значения из строк и преобразуем в int
hotels['days_since_review'] = hotels['days_since_review'].str.extract('(\d+)').astype(int)

# Проверка результата
print(hotels['days_since_review'].head())


0    531
1    203
2    289
3    681
4    516
Name: days_since_review, dtype: int32


2.2 Закодируем 'review_date' числом.

In [66]:
# Преобразуем столбец 'review_date' в формат datetime
hotels['review_date'] = pd.to_datetime(hotels['review_date'], errors='coerce')

# Вычисляем количество дней с минимальной даты в датасете или текущей даты
# Пример: разница в днях от минимальной даты
min_date = hotels['review_date'].min()
hotels['days_since_review_date'] = (hotels['review_date'] - min_date).dt.days

# Проверка результата
print(hotels[['review_date', 'days_since_review_date', 'days_since_review']].head())


  review_date  days_since_review_date  days_since_review
0  2016-02-19                     199                531
1  2017-01-12                     527                203
2  2016-10-18                     441                289
3  2015-09-22                      49                681
4  2016-03-05                     214                516


2.3 У нас есть 225 уникальных значений для 'reviewer_nationality'. Давайте присвоим им уникальные числовые значения.

In [67]:
from sklearn.preprocessing import LabelEncoder

# Инициализируем LabelEncoder
le = LabelEncoder()

# Применяем LabelEncoder к столбцу 'reviewer_nationality'
hotels['reviewer_nationality_encoded'] = le.fit_transform(hotels['reviewer_nationality'])

# Проверка результата
print(hotels[['reviewer_nationality', 'reviewer_nationality_encoded']].head())


  reviewer_nationality  reviewer_nationality_encoded
0      United Kingdom                            213
1      United Kingdom                            213
2               China                             42
3      United Kingdom                            213
4              Poland                            160


2.4 Создадим новый признак количества tags в отзыве.

In [68]:
# Шаг 1: Разделяем строку с тегами на список тегов
hotels['tags_count'] = hotels['tags'].str.split(',').apply(len)

# Проверка результата
print(hotels[['tags', 'tags_count']].head())


                                                tags  tags_count
0  [' Leisure trip ', ' Couple ', ' Studio Suite ...           5
1  [' Business trip ', ' Couple ', ' Standard Dou...           4
2  [' Leisure trip ', ' Solo traveler ', ' Modern...           5
3  [' Leisure trip ', ' Solo traveler ', ' Standa...           4
4  [' Business trip ', ' Couple ', ' Standard Dou...           4


In [69]:
# Подсчёт уникальных названий отелей
unique_hotels = hotels['hotel_address'].nunique()

print(f"Количество уникальных отелей: {unique_hotels}")


KeyError: 'hotel_address'